### TFIDF

In [1]:
import numpy as np
import jieba
import pandas as pd
from gensim import models
from gensim import corpora
from gensim import similarities
import pyLDAvis
import pyLDAvis.gensim_models

C:\Users\Tibame_T14\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [5]:
df_jiebatop40=pd.read_json('jieba_top40_1.json',lines=True)
df_jiebatop40['jieba_list']=df_jiebatop40['jieba_cut'].apply(lambda x: x.split(','))
df_jiebatop40.head()

,書名,ISBN,jieba_cut,jieba_list
0,詭圖：地圖歷史上最偉大的神話、謊言和謬誤（精裝）,9789869590235,"全彩,高度,教授,博物,荒野,副教授,排行榜,未知,困境,揭露,大地,征服,探索,主人,整理...","[全彩, 高度, 教授, 博物, 荒野, 副教授, 排行榜, 未知, 困境, 揭露, 大地,..."
1,瞄過一眼就忘不了的世界史：高中老師╳神級YouTuber 2，000萬次點閱的超人氣課程,9789863844433,"世界史,神級,考前,年表,死背,事件,高分,故事,好看,特色,分章,整理,不用,瞄過,狂銷,...","[世界史, 神級, 考前, 年表, 死背, 事件, 高分, 故事, 好看, 特色, 分章, ..."
2,忘了自己是動物的人類：重思生命起源的歷史與身而為人的意義,9789864779376,"認為,身為,台灣,起源,思考,星球,人性,狂想,故事,義荷,葡俄西,視為,將人,將身,作為,...","[認為, 身為, 台灣, 起源, 思考, 星球, 人性, 狂想, 故事, 義荷, 葡俄西, ..."
3,大戰略：耶魯大學長紅20年大師課程，從歷史提煉的領導決策心法,9789570856712,"狐狸,甘迺迪,掌握,學程,失準,政策,心法,探看,政治,全局,跨越,同事,思考,方式,原本,...","[狐狸, 甘迺迪, 掌握, 學程, 失準, 政策, 心法, 探看, 政治, 全局, 跨越, ..."
4,2021世界年鑑,9789869278799,"疫情,全球,大事,反送,肆虐,入主,肺炎,世界,深入,整理,香港,全面,控制,北京","[疫情, 全球, 大事, 反送, 肆虐, 入主, 肺炎, 世界, 深入, 整理, 香港, 全..."


In [9]:
corpus=df_jiebatop40['jieba_list'].to_list()
dictionary = corpora.Dictionary(corpus)
bow_corpus=[dictionary.doc2bow(text) for text in corpus]
print('第一本書關鍵字的ID與次數',bow_corpus[:1])

地一本書關鍵字的ID與次數 [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]]


In [12]:
tfidf=models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
for i in corpus_tfidf[:1]:
    print('第一本書的關鍵字重要性',i)

第一本書的關鍵字重要性 [(0, 0.23717670284476003), (1, 0.22074846140844331), (2, 0.27836221302529307), (3, 0.3866341835561467), (4, 0.22850677141578762), (5, 0.25953457918341005), (6, 0.2662974383239476), (7, 0.23269223906089254), (8, 0.1763416406279392), (9, 0.23554027507468184), (10, 0.1837166009395621), (11, 0.1596922670212601), (12, 0.255312147705922), (13, 0.22005136315454046), (14, 0.32299223750629014), (15, 0.24423837339102564)]


### Adding LDA

In [13]:
# 35主題 , 50主題 overfitting 嚴重
lda_model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=35,passes=3)

In [14]:
lda_model.print_topics()

[(34,
  '0.033*"教室" + 0.020*"避免" + 0.019*"治理" + 0.016*"叛逆" + 0.015*"眾生" + 0.013*"恐怖份子" + 0.013*"啟程" + 0.013*"村落" + 0.011*"不容" + 0.011*"通向"'),
 (28,
  '0.031*"能力" + 0.028*"技巧" + 0.024*"不能" + 0.019*"打算" + 0.019*"找到" + 0.018*"提升" + 0.015*"衍生" + 0.014*"品牌" + 0.013*"人生" + 0.013*"生死"'),
 (27,
  '0.059*"魔法" + 0.039*"妖怪" + 0.035*"王者" + 0.034*"武器" + 0.027*"好像" + 0.024*"黑手" + 0.022*"人能" + 0.022*"逃出" + 0.018*"代替" + 0.016*"父子"'),
 (2,
  '0.094*"尺寸" + 0.024*"女子" + 0.024*"阻止" + 0.021*"普通" + 0.017*"作品" + 0.015*"客人" + 0.012*"人生" + 0.012*"名作" + 0.011*"享受" + 0.011*"眼看"'),
 (32,
  '0.025*"改善" + 0.023*"逼近" + 0.023*"性命" + 0.021*"文豪" + 0.021*"男主角" + 0.020*"友情" + 0.020*"家人" + 0.019*"犯人" + 0.018*"克服" + 0.017*"觀眾"'),
 (16,
  '0.041*"富人" + 0.032*"模型" + 0.029*"基本" + 0.028*"情境" + 0.027*"化為" + 0.017*"使用者" + 0.016*"量化" + 0.015*"火焰" + 0.014*"判定" + 0.013*"光影"'),
 (14,
  '0.071*"美女" + 0.054*"案件" + 0.046*"不到" + 0.046*"使出" + 0.039*"不用" + 0.033*"程度" + 0.020*"接踵" + 0.014*"新任" + 0.011*"查理" + 0.010*"死因"'),
 (3,
  '0.065*"拯救

In [15]:
corpus_lda = lda_model[corpus_tfidf]

In [16]:
overview=pyLDAvis.gensim_models.prepare(lda_model,bow_corpus,dictionary) # pyLDAvis -> (model,[書籍id,次數],不重複之文字)

In [17]:
# show 圖形化介面
pyLDAvis.display(overview) 

In [18]:
index=similarities.MatrixSimilarity(corpus_lda)

In [34]:
df_jiebatop40.head()

,書名,ISBN,jieba_cut,jieba_list
0,詭圖：地圖歷史上最偉大的神話、謊言和謬誤（精裝）,9789869590235,"全彩,高度,教授,博物,荒野,副教授,排行榜,未知,困境,揭露,大地,征服,探索,主人,整理...","[全彩, 高度, 教授, 博物, 荒野, 副教授, 排行榜, 未知, 困境, 揭露, 大地,..."
1,瞄過一眼就忘不了的世界史：高中老師╳神級YouTuber 2，000萬次點閱的超人氣課程,9789863844433,"世界史,神級,考前,年表,死背,事件,高分,故事,好看,特色,分章,整理,不用,瞄過,狂銷,...","[世界史, 神級, 考前, 年表, 死背, 事件, 高分, 故事, 好看, 特色, 分章, ..."
2,忘了自己是動物的人類：重思生命起源的歷史與身而為人的意義,9789864779376,"認為,身為,台灣,起源,思考,星球,人性,狂想,故事,義荷,葡俄西,視為,將人,將身,作為,...","[認為, 身為, 台灣, 起源, 思考, 星球, 人性, 狂想, 故事, 義荷, 葡俄西, ..."
3,大戰略：耶魯大學長紅20年大師課程，從歷史提煉的領導決策心法,9789570856712,"狐狸,甘迺迪,掌握,學程,失準,政策,心法,探看,政治,全局,跨越,同事,思考,方式,原本,...","[狐狸, 甘迺迪, 掌握, 學程, 失準, 政策, 心法, 探看, 政治, 全局, 跨越, ..."
4,2021世界年鑑,9789869278799,"疫情,全球,大事,反送,肆虐,入主,肺炎,世界,深入,整理,香港,全面,控制,北京","[疫情, 全球, 大事, 反送, 肆虐, 入主, 肺炎, 世界, 深入, 整理, 香港, 全..."


### 驗證LDA

In [76]:
idx=123457
one_book=df_jiebatop40.iloc[idx,3]

In [77]:
query_bow=dictionary.doc2bow(one_book)

In [78]:
query_lda=lda_model[query_bow]

In [79]:
query_lda

[(4, 0.5072105), (31, 0.25703558)]

In [80]:
sims=index[query_lda]

In [81]:
max5=list(sims.argsort()[::-1][0:5])

In [82]:
max5

[239675, 72301, 406767, 93550, 310748]

In [84]:
for i in max5:
    print(sims[i])

0.9999999
0.9991412
0.9970605
0.99684525
0.9954294


In [85]:
df_jiebatop40.iloc[max5,:2]

,書名,ISBN
239675,愛與生存的勇氣,9789867349255
72301,走進圖畫故事書的世界(B6D),4711158890035
406767,在這孤立的海島上,9789570494068
93550,親愛的獅子爺爺,9789865863975
310748,法語生詞攻略篇(精裝),9789628320387


### 不合適結論

原因:詞袋(corpus) 過於稀疏，出現overfitting<br>

#### 預期解決方式: 
+ 增加jieba_cut文字數，增加磁袋<br>
+ 以主題數觀察合理analyse應調整設定多少個keyword<br>